In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


2023-07-19 04:37:58.812957: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# Veri setini yükleme
data_path = "data/captchas.csv"
df = pd.read_csv(data_path)
column_names = ['Image_Path', 'Caption']  # Başlık sütunlarının listesini oluşturun

df.columns = column_names  # DataFrame'e yeni başlık sütunlarını atayın



In [3]:

# Resim dosyalarının konumları ve metinleri alın
image_paths = df["Image_Path"].values
captions = df["Caption"].values

In [4]:


# Resimleri yükleme ve ön işleme
def load_and_preprocess_image(image_path):
    image = keras.preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=(200, 70))
    image = keras.preprocessing.image.img_to_array(image) / 255.0
    return image

In [5]:

images = np.array([load_and_preprocess_image(path) for path in image_paths])

# Metinleri vektörleştirme
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(captions)
captions_encoded = tokenizer.texts_to_sequences(captions)
captions_encoded = keras.preprocessing.sequence.pad_sequences(captions_encoded)


In [6]:

# Veri kümesini eğitim ve test kümelerine ayırma
train_images, test_images, train_captions, test_captions = train_test_split(
    images, captions_encoded, test_size=0.25, random_state=42
)

In [7]:
# Modeli oluşturma
image_input = layers.Input(shape=(200, 70, 1), name="image_input")
image_conv = layers.Conv2D(32, (3, 3), activation="relu")(image_input)
image_pool = layers.MaxPooling2D((2, 2))(image_conv)
image_flat = layers.Flatten()(image_pool)

caption_input = layers.Input(shape=(captions_encoded.shape[1],), name="caption_input")
caption_embedding = layers.Embedding(len(tokenizer.word_index) + 1, 50)(caption_input)
caption_flat = layers.Flatten()(caption_embedding)

concatenated = layers.concatenate([image_flat, caption_flat])
hidden = layers.Dense(64, activation="relu")(concatenated)
output = layers.Dense(captions_encoded.shape[1], activation="softmax")(hidden)

model = keras.models.Model(inputs=[image_input, caption_input], outputs=output)

In [8]:
# Modeli derleme
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [10]:
# Modeli eğitme
history = model.fit(
    {"image_input": train_images, "caption_input": train_captions},
    train_captions[:,-1],
    batch_size=32,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10


2023-07-19 04:42:49.458541: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 55148544 exceeds 10% of free system memory.
2023-07-19 04:42:49.704856: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 55148544 exceeds 10% of free system memory.


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [ ]:
# Modelin performansını değerlendirme
test_loss, test_accuracy = model.evaluate({"image_input": test_images, "caption_input": test_captions}, test_captions)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
